# Майнор "Интеллектуальный анализ данных"

# Курс "Прикладные задачи анализа данных"

## Лабораторная работа №2. RecSys

В рамках данной лабораторной работы вам предлагается решить задачу рекомендательной системы на основе следующих данных:

* **rating.csv** - рейтинги аниме по 10 бальной шкале
* **anime.csv** - описание items

Скачать данные можно [здесь](https://drive.google.com/drive/u/1/folders/1FarHUuqQq4tQSlERB9K8uEfZAZT9tQWv)

# Данные

Данные содержат информацию об предподчтениях 73,516 пользователей в 12,294 аниме. Каждый пользователь может добавить аниме и выставить рейтниг.

Anime.csv

* anime_id - идентификатор аниме (items)
* name - полное название аниме
* genre - категория/категории для аниме (разделены запятой).
* type - тип: movie, TV, OVA, etc.
* episodes - количество эпизодов (для экранизаций)
* rating - средний рейтинг по аниме.
* members - количество членов коммьюнити к аниме

Rating.csv

* user_id - идентификатор пользователя (users)
* anime_id - идентификатор аниме
* rating - рейтинг пользователя по аниме (-1 пользователь смотрел, но не оставил рейтинг).

### ``` Если вычеслительные ресурсы не позволяют решить задачу на всех данных, то необходимо это сделать на "сэмпле" данных (выбрав от 5% ids' для user и item). Этот же подход справедлив и для оценки работы алгоритомв```

## Необходимо сделать:

### В данной лабораторной работе я буду рассматривать только явное взаимодействие (указание рейтинга аниме)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
anime = pd.read_csv("../input/anime-recommendations-database/anime.csv", index_col=None)

In [ ]:
rating = pd.read_csv("../input/anime-recommendations-database/rating.csv", index_col=None)

In [ ]:
display(anime.shape)
display(rating.shape)

In [ ]:
anime.head()

In [ ]:
rating.head()

Рассмотрим количество неявного рейтинга

In [ ]:
rating[rating['rating'] == -1].shape[0] / rating.shape[0]

19% - достаточно много, несмотря на это удалим все данные с неявным указанием рейтинга

In [ ]:
rating = rating[rating['rating'] != -1]

### Задание №1 : Exploratory analysis (2 балла):

* Распределение числа пользователей по количеству взаимодействий

Сколько аниме люди оценивают в среднем

In [ ]:
quantile = rating.groupby(['user_id']).size().quantile(0.95)
quant_rating = rating.groupby(['user_id']).size().where(lambda x: x <= quantile).dropna()
print('modes', quant_rating.mode().values)
print('mean', quant_rating.mean())

In [ ]:
s = rating.groupby(['user_id']).count()['anime_id'].to_frame(0).groupby(0)[0].count()

In [ ]:
res = s.to_frame(0)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
fig.text(0.5, 0.04, 'Число взаимодействий', ha='center')
fig.text(0.04, 0.5, 'Число пользователей', va='center', rotation='vertical')
ax1.plot(res)
ax2.plot(res[res > 10])
ax3.plot(res[res > 100])
ax4.plot(res[res > 500])

На графиках видно, что чаще всего люди оценивают всего 1 фильм, а в среднем 67 фильмов

* Разпределение числа "айтемов" по количеству взаимодействий

In [ ]:
quantile = rating.groupby(['anime_id']).size().quantile(0.95)
quant_rating = rating.groupby(['anime_id']).size().where(lambda x: x <= quantile).dropna()
print('modes', quant_rating.mode().values)
print('mean', quant_rating.mean())

In [ ]:
s = rating.groupby(['anime_id']).count()['user_id'].to_frame(0).groupby(0)[0].count()

In [ ]:
res = s.to_frame(0)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
fig.text(0.5, 0.04, 'Число взаимодействий', ha='center')
fig.text(0.04, 0.5, 'Число айтемов', va='center', rotation='vertical')
ax1.plot(res)
ax2.plot(res[res > 10])
ax3.plot(res[res > 50])
ax4.plot(res[res > 100])

Видно, что в среднем фильм имеет 311 взаимодействий, но много фильмов всего с несколькими взаимодействиями

* Распределение числа рейтингов, средних рейтингов по пользователям, по "айтемам"


Распределение числа рейтингов

In [ ]:
res = rating.groupby('rating').size()

In [ ]:
plt.bar(res.index.tolist(), res.tolist())
plt.xlabel('Оценка')
plt.ylabel('Число оценок')

Видно, что в основном люди ставят положительные оценки 7 и выше, 8 - самая частая оценка

Распределение средних рейтингов по пользователям

In [ ]:
res = rating.groupby('user_id')['rating'].mean().apply(lambda x: round(x)).to_frame(0).groupby(0)[0].count()

In [ ]:
plt.bar(res.index.tolist(), res.tolist())
plt.xlabel('Средняя оценка')
plt.ylabel('Число оценок')

Видно, что пользователи в среднем ставят положительные оценки > 7, однако есть незначительный процент людей, которые ставят в основном негативные оценки (возможно только негативные)

Распределение средних рейтингов по айтемам

In [ ]:
res = rating.groupby('anime_id')['rating'].mean().apply(lambda x: round(x)).to_frame(0).groupby(0)[0].count()

In [ ]:
plt.bar(res.index.tolist(), res.tolist())
plt.xlabel('Средняя оценка')
plt.ylabel('Число фильмов')

Видно, что средний рейтинг фильмов от 6 до 8

* и т.д. (бонус)

Рассмотрим nan значения

In [ ]:
rating[rating.isna().any(axis=1)].shape[0]

В rating нет nan значений

In [ ]:
anime[anime.isna().any(axis=1)].shape[0]

Есть 277 строк с nan значениями

Заменяем nan рейтинг на средний из датасета

In [ ]:
def calculate_rating(dataset, anime_id, default_value = -1):
    return dataset[dataset['anime_id'] == anime_id]['rating'].mean()

In [ ]:
anime.loc[anime['rating'].isna(), ['rating']] = anime[anime['rating'].isna()].apply(lambda x: calculate_rating(rating, x['anime_id']), axis = 1)

Заменяем nan жанр на Unknown

In [ ]:
anime.loc[anime['genre'].isna(), ['genre']] = "Unknown"

Убираем все остальные строки с nan. Их нет в rating

In [ ]:
anime = anime.dropna()

In [ ]:
def split_column(col, delim = ', '):
    return col.split(delim)

Распределение количества аниме по жанрам

In [ ]:
anime['genre'] = anime['genre'].apply(split_column) 

In [ ]:
anime.explode('genre').groupby('genre').size().sort_values(ascending = False)

Видно, что самым популярным оцениваемым жанром является комедия, а хентай редко оценивают ^_^

In [ ]:
my_dict = {}

for genre_list in anime['genre'].to_list():
    for genre in genre_list:
        other_genres = [g for g in genre_list if g != genre]
        if (genre not in my_dict):
            my_dict[genre] = {}
        
        for og in other_genres:
            if my_dict[genre].get(og):
                my_dict[genre][og] += 1
            else:
                my_dict[genre][og] = 1


In [ ]:
def get_top_n_genre_correlations(genre, n):
    return sorted(my_dict[genre], key= my_dict[genre].get, reverse = True)[:n]

In [ ]:
get_top_n_genre_correlations('Drama', 5)

Распределение аниме по типу

In [ ]:
anime.groupby('type').size().sort_values(ascending = False)

Распределение по числу эпизодов

In [ ]:
anime.groupby('episodes').size().sort_values(ascending = False)

Распределение по числу слов в названии

In [ ]:
anime2 = anime.copy()

In [ ]:
anime2['name'].apply(lambda x: len(x.split(' ')))

In [ ]:
anime2['name_token_count'] = anime2['name'].apply(lambda x: len(x.split(' ')))

In [ ]:
anime2.groupby('name_token_count').size()

Есть фильмы, состоящие из > 16 слов... Интересно, как они называются 😀

In [ ]:
anime2[anime2['name_token_count'] > 16]['name'].to_list()

### Задание №2 : Оценить разреженность данных по рейтингу (1/2 балла)

Число уникальных аниме: 

In [ ]:
rating.shape

In [ ]:
anime_count = rating.groupby('anime_id').nunique().shape[0]

In [ ]:
people_count = rating.groupby('user_id').nunique().shape[0]

In [ ]:
total_size = people_count * anime_count

In [ ]:
real_size = rating.shape[0]

In [ ]:
print("anime_count {}".format(anime_count))
print("people_count {}".format(people_count))
print("total_size {}".format(total_size))
print("real_size {}".format(real_size))
print("sparcity_of_matrix {}".format(1 - real_size / total_size))

Очень высокая разреженность данных: 99%, так как пользователи в основном оценивают малое количетсво аниме 

### Задание №3 : Разделить данные на тренировочные и валидационные (1/2 балла)

Разделим таблицу rating на тренировочные и тестовые данные по пользователям, предварительно отфильтровав пользователей с количеством проставленных оценок, меньшим определенного порога.

In [ ]:
TRESHOLD = 0.7
MIN_RATE = 20

In [ ]:
old_len = rating.shape[0]

In [ ]:
rating = rating[rating['user_id'].groupby(rating['user_id']).transform('size') > MIN_RATE]

Сколько данных осталось

In [ ]:
rating.shape[0] / old_len

Отфильтровали 3% пользователей

Сколько оперативной памяти в гб будет занимать полный датасет:

In [ ]:
rating['anime_id'].nunique() * rating['user_id'].nunique() * 32 / 1024 / 1024 / 1024

In [ ]:
from sklearn.model_selection import train_test_split

Параметр stratify определяет, по какому признаку пропорционально разделять данные. В данном случае разделяем, чтобы в train и test попали 0.7 и 0.3 рейтингов от пользователя соответственно

In [ ]:
train, test = train_test_split(rating, test_size = 1 - TRESHOLD, stratify = rating['user_id'])

In [ ]:
display(train['user_id'].nunique(),test['user_id'].nunique())

In [ ]:
u_id = train.iloc[0]['user_id']

In [ ]:
trainsz = train[train['user_id'] == u_id].shape[0]

In [ ]:
testsz = test[test['user_id'] == u_id].shape[0]

In [ ]:
trainsz / (trainsz + testsz)

### Задание №4 : Решить задачу на основе предложения всем юзерам наиболее популярных item (1 балл)

In [ ]:
from abc import abstractmethod

In [ ]:
class BaseTopNPredictorClass():
    @abstractmethod
    def predict(self, user_id: int, top_n: int, remove_seen: list) -> list:
        pass

Предложим топ 10 самых популярных аниме по взаимодействиям пользователей

In [ ]:
anime[anime['anime_id'].isin(rating.groupby(['anime_id']).size().sort_values(ascending = False).head(10).index)].sort_values('members', ascending = False)

In [ ]:
class AnimeRatingPredictorClass(BaseTopNPredictorClass):
    def __init__(self, df, item_col):
        self.val = df.groupby([item_col]).size().sort_values(ascending = False).index
        
    def predict(self, user_id: int, top_n: int, remove_seen = []) -> list:
        tmp = self.val
        return list(tmp[~tmp.isin(remove_seen)])[:top_n]

In [ ]:
arpc = AnimeRatingPredictorClass(rating, 'anime_id')
arpc.predict(1, 10)

Или предложим топ 10 самых популярных аниме из количества человек в комьюнити

In [ ]:
anime.sort_values('members', ascending = False).head(10)

In [ ]:
class AnimeTopNByColumnPredictorClass(BaseTopNPredictorClass):
    def __init__(self, df, item_col, column):
        self.val = df.sort_values(column, ascending = False)[item_col]
        
    def predict(self, user_id: int, top_n: int, remove_seen = []) -> list:
        tmp = self.val
        return list(tmp[~tmp.isin(remove_seen)])[:top_n]

In [ ]:
atnbcpc = AnimeTopNByColumnPredictorClass(anime, 'anime_id', 'members')
atnbcpc.predict(1, 10)

Или предложим топ 10 самых популярных аниме по взвешенному рейтингу

In [ ]:
m = anime['members'].quantile(0.75)
c = anime['rating'].mean()

In [ ]:
def calc_weighted_score(x, c = c, m = m):
    v = x['members']
    r = x['rating']
    return (v/(v+m)*r)+(m/(v+m)*c)

In [ ]:
qualified_anime = anime.copy().loc[anime['members']>m]
qualified_anime['score'] = calc_weighted_score(qualified_anime)
qualified_anime.sort_values('score', ascending = False).head(10)

In [ ]:
class AnimeTopNWeightedRatingPredictorClass(BaseTopNPredictorClass):
    def __init__(self, df, item_col, m_col, c_col):
        m = df[m_col].quantile(0.75)
        c = df[c_col].mean()

        qualified = df.copy().loc[df[m_col]>m]
        qualified['score'] = calc_weighted_score(qualified)
        self.val = qualified.sort_values('score', ascending = False)[item_col]
        
    def predict(self, user_id: int, top_n: int, remove_seen = []) -> list:
        tmp = self.val
        return list(tmp[~tmp.isin(remove_seen)])[:top_n]

In [ ]:
atnwrpc = AnimeTopNWeightedRatingPredictorClass(anime, 'anime_id', 'members', 'rating')
atnwrpc.predict(1, 10)

### Задание №5 : Решить задачу на основе коллоборативной фильтрации (2 балла)

* Реализовать один из методов коллоборативной фильтрации SVD, SVD++, ALS, ALS with implicit feedback

Решим задачу с помощью SVD

In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD

In [ ]:
class AnimeSVDPredictorClass(BaseTopNPredictorClass):
    def __init__(self, n_factors = 10, n_epochs = 30, svd = None, remove_seen = False):
        self.svd = svd if svd is not None else SVD(random_state=0, n_factors = n_factors, n_epochs = n_epochs, verbose=True)
        self.item_ids = None
        self.predictions = {}
        
    def set_item_ids(self, item_ids: list):
        self.predictions = {}
        self.item_ids = item_ids
        
    def fit(self, train_df):
        train_sup = Dataset.load_from_df(train_df,  reader = Reader(line_format='user item rating'))
        train_sup = train_sup.build_full_trainset()
        self.svd.fit(train_sup)
        
    def predict(self, user_id: int, top_n: int, remove_seen = []) -> list:
        if (self.item_ids is None):
            raise Exception('item_ids not found')
            
        predictions = [[item_id, self.svd.predict(user_id, item_id, clip = False).est] for item_id in self.item_ids]
        predictions = sorted(predictions, key = lambda x: x[1], reverse = True)
        predictions = map(lambda x: x[0], predictions)
        
        return list(filter(lambda x: x not in remove_seen, predictions))[:top_n]
    
    def predict_and_save(self, user_id: int, remove_seen = []) -> list:
        predictions = self.predict(user_id, None, remove_seen)
        self.predictions[user_id] = list(filter(lambda x: x not in remove_seen, predictions))
        
        
    def predict_score(self, user_id: int, item_id: int) -> float:
        return self.svd.predict(user_id, item_id, clip = False).est

In [ ]:
asvdpc = AnimeSVDPredictorClass(n_epochs = 10)
asvdpc.fit(train)

In [ ]:
asvdpc.set_item_ids(list(test['anime_id'].unique()))

In [ ]:
asvdpc.predict(3, 10)

In [ ]:
[asvdpc.predict_score(3, item_id) for item_id in asvdpc.predict(3, 5)]

In [ ]:
tmp = test[test['user_id'] == 3]

In [ ]:
tmp['pred_score'] = [asvdpc.predict_score(3, item_id) for item_id in tmp['anime_id']]

In [ ]:
tmp

In [ ]:
plt.bar(range(0, tmp.shape[0]), tmp['rating'], label = 'real')
plt.bar(range(0, tmp.shape[0]), tmp['pred_score'], label = 'pred')
plt.legend()

Видно, что алгоритм с большой долей ошибки определяет рейтинг фильмов, однако в  большинстве случаев отражает тренды

### Задание №6 Решить задачу на основе контент-based подхода, выбрав один из методов решения (2 балла):

* На основание векторов с факторизационной матрицы
* При помощи "ембеддингов"  item'ов

Попробуем решить задачу методом ембеддингов 

Для этого составим вектора из anime.name и определим названия, наиболее близкие к понравившимся и наиболее далекие к непонравившимся фильмам

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

In [ ]:
from gensim.models import Word2Vec

Пробовал делать препроцессинг - в данной задаче это не нужно

In [ ]:
# import nltk
# from collections import Counter
# import unicodedata
# import sys
# import re
# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()
# remove_punct_map = dict.fromkeys([i for i in range(sys.maxunicode)
#                                  if unicodedata.category(chr(i)).startswith('P')], ' ')
# stopwords = set(nltk.corpus.stopwords.words('english'))
# def text_cleaning(text):
#     text = re.sub(r'&quot;', '', text)
#     text = text.lower()
#     text = re.sub(r'.hack//', '', text)
#     text = re.sub(r'&#039;', '', text)
#     text = re.sub(r'A&#039;s', '', text)
#     text = re.sub(r'I&#039;', 'I\'', text)
#     text = re.sub(r'&amp;', 'and', text)
#     text = text.translate(remove_punct_map)
#     tokens = list(map(lemmatizer.lemmatize, nltk.word_tokenize(text)))
#     tokens = list(filter(lambda token: token not in stopwords and token != 'nu', tokens))
    
#     return ' '.join(tokens)

# anime['clean_name'] = anime['name'].apply(text_cleaning)

In [ ]:
corpus = list(anime.merge(train, on = ['anime_id']).groupby('user_id')['name'].agg(list))
w2v_model = Word2Vec(min_count=1, workers=cores-1, vector_size = 10)
w2v_model.build_vocab(corpus, progress_per=1000)

In [ ]:
corpus[0][:5]

In [ ]:
%%time
w2v_model.train(corpus, total_examples=w2v_model.corpus_count, epochs=10, report_delay=1)

In [ ]:
class AnimeW2VPredictorClass(BaseTopNPredictorClass):
    def __init__(self, w2v, film_descr_df, rating_df, user_id_col = 'user_id', item_id_col = 'anime_id', rating_col = 'rating', emb_feature_col = 'name'):
        self.rating_df = rating_df
        self.film_descr_df = film_descr_df
        self.user_id_col = user_id_col
        self.item_id_col = item_id_col
        self.rating_col = rating_col
        self.emb_feature_col = emb_feature_col
        self.w2v = w2v
        
    def predict(self, user_id: int, top_n: int, remove_seen = []) -> list:
        # Те, что >= своей средней оценки оценил пользователь
        positive_ids = self.rating_df[(self.rating_df[self.user_id_col] == user_id) & (self.rating_df[self.rating_col].ge(self.rating_df[self.rating_df[self.user_id_col] == user_id][self.rating_col].mean() - 1))][self.item_id_col]
        negative_ids = self.rating_df[(self.rating_df[self.user_id_col] == user_id) & (~self.rating_df[self.item_id_col].isin(positive_ids))][self.item_id_col]
        pos_anime_names = self.film_descr_df[self.film_descr_df[self.item_id_col].isin(positive_ids)][self.emb_feature_col]
        neg_anime_names = self.film_descr_df[self.film_descr_df[self.item_id_col].isin(negative_ids)][self.emb_feature_col]
        topn = len(self.w2v.wv.index_to_key)
        names = list(map(lambda x: x[0], self.w2v.wv.most_similar(positive = pos_anime_names, negative = neg_anime_names, topn = topn)))
        predicted = pd.DataFrame(list(enumerate(names)), columns = ['idx', self.emb_feature_col])
        predicted = self.film_descr_df[self.film_descr_df[self.emb_feature_col].isin(names)].merge(predicted, on = [self.emb_feature_col]).sort_values('idx')[self.item_id_col]
        return predicted[~predicted.isin(remove_seen)].to_list()[:top_n]

In [ ]:
cb = AnimeW2VPredictorClass(w2v_model, anime, rating)

In [ ]:
%%time
cb.predict(3, 10)

### Задание №7 Оценить работу алгоритмов выбрав одну из метрик, сделать вывод по результатам работы (2 балла):

* MAP@k
* NDCG@k
* MRR

In [ ]:
def mean_reciprocal_rank(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])


def r_precision(r):
    r = np.asarray(r) != 0
    z = r.nonzero()[0]
    if not z.size:
        return 0.
    return np.mean(r[:z[-1] + 1])


def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)


def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)


def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])


def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [ ]:
from time import time
def get_ranking_table(algo, train, test, user_id_colname = 'user_id', item_id_colname = 'anime_id', rating_colname = 'rating'):
    preds_lst = []
    start = time()
    finish = None
    for i, user_id in enumerate(train[user_id_colname].unique()):
        seen = list(train[train[user_id_colname] == user_id][item_id_colname])
        preds_lst.append([user_id, algo.predict(user_id, None, seen)])    
        if (not finish):
            finish = time()
            print(finish - start, 'per operation')
            
        if (i % 200 == 0):
            print (i)
            
    predictions = pd.DataFrame(data=preds_lst, columns=[user_id_colname, 'predicted'])
    cold_items = [i for i in test[item_id_colname].unique() if i not in train[item_id_colname].unique()]
    print('len of cold_items = ', len(cold_items))
    
    user_preferences = test.merge(train, on = [user_id_colname, item_id_colname], how='left', suffixes=('', '_TrainRating'))
    user_preferences = user_preferences[~user_preferences[user_id_colname].isin(cold_items)]
    user_preferences = user_preferences.sort_values(rating_colname, ascending = False).groupby([user_id_colname])[item_id_colname].apply(lambda x: list(x)).to_frame().reset_index()
    user_preferences.rename(columns = {item_id_colname: 'real'}, inplace = True)
    ranking_table = user_preferences.merge(predictions, on = [user_id_colname], how='left')

    return ranking_table

In [ ]:
def get_top_n(df, n, item_real_col_name, item_pred_col_name):
    return df.apply(lambda x: [int(pred in x[item_real_col_name][:n]) for pred in x[item_pred_col_name][:n]], axis=1)

In [ ]:
def get_metrics(ranking_table, top_n):
    tmp_df = ranking_table.copy()
    score_results = pd.DataFrame(columns=['MRR', 'MAP@K', 'NDCG@k'])
    for n in top_n:
        topn_res = get_top_n(tmp_df, n, 'real', 'predicted')
        score_results.loc['top_'+str(n), 'MAP@K'] = mean_reciprocal_rank(list(topn_res.values))
        score_results.loc['top_'+str(n), 'MRR'] = mean_average_precision(list(topn_res.values))
        score_results.loc['top_'+str(n), 'NDCG@k'] = np.mean([ndcg_at_k(i, n) for i in list(topn_res.values)])
    return score_results

Так как предсказание фильмов для всех пользователей занимает значительное время и большое количество оперативной памяти, возьмем только 5% рандомных user_id

In [ ]:
filtered_user_ids = pd.DataFrame(train['user_id'].unique()).sample(frac = 0.05, random_state = 42)[0].to_list()

In [ ]:
min_train = train[train['user_id'].isin(filtered_user_ids)]

In [ ]:
min_test = test[test['user_id'].isin(filtered_user_ids)]

In [ ]:
min_asvdpc = AnimeSVDPredictorClass(n_epochs = 30)

In [ ]:
min_asvdpc.fit(min_train)

In [ ]:
min_asvdpc.set_item_ids(list(min_train['anime_id'].unique()))

In [ ]:
min_asvdpc_ranking_table = get_ranking_table(min_asvdpc, min_train, min_test)

Результат работы алгоритма SVD

In [ ]:
get_metrics(min_asvdpc_ranking_table, [1, 3, 10, 100])

In [ ]:
min_arpc = AnimeRatingPredictorClass(min_train, 'anime_id')
arpc_ranking_table = get_ranking_table(min_arpc, min_train, min_test)

Результат предложений фильмов по топ баллам из rating

In [ ]:
get_metrics(arpc_ranking_table, [1, 3, 10, 100])

In [ ]:
min_atnbcpc = AnimeTopNByColumnPredictorClass(anime, 'anime_id', 'members')
atnbcpc_ranking_table = get_ranking_table(min_atnbcpc, min_train, min_test)

Результат предложений фильмов по рейтингу из датасета anime

In [ ]:
get_metrics(atnbcpc_ranking_table, [1, 3, 10, 100])

In [ ]:
min_atnwrpc = AnimeTopNWeightedRatingPredictorClass(anime, 'anime_id', 'members', 'rating')
atnwrpc_ranking_table = get_ranking_table(min_atnwrpc, min_train, min_test)

Результат работы предложений по взвешенной сумме

In [ ]:
get_metrics(atnwrpc_ranking_table, [1, 3, 10, 100])

In [ ]:
corpus = list(anime.merge(min_train, on = ['anime_id']).groupby('user_id')['name'].agg(list))
min_w2v_model = Word2Vec(min_count=1, workers=cores-1, vector_size = 10)
min_w2v_model.build_vocab(corpus, progress_per=1000)

In [ ]:
%%time
min_w2v_model.train(corpus, total_examples=min_w2v_model.corpus_count, epochs=10, report_delay=1)

In [ ]:
min_cb = AnimeW2VPredictorClass(min_w2v_model, anime, min_train)

In [ ]:
min_cb_ranking_table = get_ranking_table(min_cb, min_train, min_test)

Результат работы content-based подхода

In [ ]:
get_metrics(min_cb_ranking_table, [1, 3, 10, 100])

#### Вывод:
Сравнив алгоритмы можно сделать вывод, что наилучшим образом показали себя dummy модели, дающие на выходе наиболее рейтинговые фильмы. Алгоритм svd разместился посередине, а предсказываение с использованием эмбеддингов айтемов дало наихудший результат.

Скорее всего, при выборе фильма и его оценке, люди обращают наибольшее внимание на рейтинг фильма, указанный на сайте и популярность фильма, чем на другие факторы.

Также можно предположить, что на всем датасете ситуация исправится в пользу алгоритма сингулярного разложения, и, возможно, стоит применить svd++ ( пробовал, оказался в 100 раз медленнее чем svd ). Однако на больших данных обучение занимает слишком много времени.

![](https://i.ytimg.com/vi/_mD2Ue4_Ya0/hqdefault.jpg)

``` Бонус (1 балл) - дополнительные графики и характеристики в EDA и правильно сделанные выводы```

```Дедлайн - 13 июня 23:59```